In [8]:
import sys
import os
sys.path.append(os.path.abspath("../../..")) 
from utils.classes.StringBuilder import StringBuilder


sb = StringBuilder()
sb.append('aaa')
print(sb.__str__())

aaa


In [ ]:
import logging
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
import os
sys.path.append(os.path.abspath("../../.."))  # Adds the project root to sys.path
os.environ['OPENBLAS_NUM_THREADS'] = '1'
import numpy as np 
import math
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
import implicit
from implicit.nearest_neighbours import bm25_weight
from sklearn.model_selection import train_test_split
import pprint

logging.basicConfig(level=logging.INFO)
log = logging.getLogger("implicit")

from recommenders.utils.shared import *

In [2]:
save_interaction_values("../../../../pa-data/Interactions.xlsx")

In [ ]:
SKU = 0
STOCK = 1
STATUS = 2
VISIBILITY = 3
ACTIVE = 4

In [3]:
Y, R, num_products = get_data()
sparse_product_user = csr_matrix(Y)
sparse_product_user = bm25_weight(sparse_product_user, K1=100, B=0.8)
# get the transpose since the most of the functions in implicit expect (user, product) sparse matrices instead of (product, user)
sparse_user_product = sparse_product_user.T.tocsr()

products = load_csv_list('Products.csv')
product_names = np.array([row[0] for row in products])
product_indexes_to_filter = [
    idx for idx, x in enumerate(products)
    if x[STOCK] == '0' or x[STATUS] != 'Published' or x[VISIBILITY] != 'Public' or x[ACTIVE] != '1'
]
print(product_indexes_to_filter)
user_ids = load_csv_list('Users.csv')[0]

Y (117, 4) R (117, 4)
num_products 117
num_users 4
[0, 1, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 51, 52, 53, 54, 56, 58, 59, 60, 62, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 103, 104, 106, 108, 109, 111, 112, 113, 114, 115, 116]


In [4]:
# https://github.com/benfred/implicit/issues/281
model = implicit.als.AlternatingLeastSquares(factors=100, regularization=0.1, alpha=1.0, iterations=128, calculate_training_loss=True)
model.fit(sparse_user_product, show_progress=True)

  0%|          | 0/128 [00:00<?, ?it/s]

INFO:implicit:Final training loss 0.0101


In [ ]:
recommendation_result_dict = get_recommendation_result_dict(model, sparse_user_product, user_ids, product_names)

In [11]:
pprint.pprint(recommendation_result_dict['47608'])

['1600Z00038',
 '2608644374',
 '2608690127',
 '2608584100',
 'SIG 048L7R',
 '90 70 220',
 '97 51 10',
 'VILL 060108',
 'KWB 49427140',
 'HT8G330']
